In [1]:
# Importações básicas
import numpy as np  #trabalhar matrizes
import pandas as pd #dataframes
import matplotlib.pyplot as plt #gráficos
from gensim.models import Word2Vec  #modelo de vetorização
import gensim.downloader as api 
from sklearn.manifold import TSNE #framework de treinamento
import re
import nltk #trabalho de frases e palavras
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


In [4]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

Exemplo de texto original:
Este filme é incrível, adorei a atuação do protagonista

Depois do pré-processamento:
['filme', 'incrível', 'adorei', 'atuação', 'protagonista']


In [6]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)

In [7]:
# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

In [ ]:
print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")


Modelo treinado com {'filme': 0, 'história': 1, 'roteiro': 2, 'desperdicei': 3, 'belíssima': 4, 'cinematografia': 5, 'fraco': 6, 'talentosos': 7, 'atores': 8, 'assistindo': 9, 'tempo': 10, 'péssimo': 11, 'assistir': 12, 'envolvente': 13, 'espetacular': 14, 'fotografia': 15, 'direção': 16, 'protagonista': 17, 'atuação': 18, 'adorei': 19, 'incrível': 20, 'recomendo': 21, 'final': 22, 'chorei': 23, 'previsível': 24, 'emocionante': 25, 'ponta': 26, 'tecnologia': 27, 'impressionantes': 28, 'especiais': 29, 'efeitos': 30, 'fim': 31, 'início': 32, 'entediante': 33, 'gostei': 34, 'cenas': 35, 'perfeitamente': 36, 'combina': 37, 'sonora': 38, 'trilha': 39, 'desenvolvidos': 40, 'mal': 41, 'personagens': 42, 'cinema': 43} palavras no vocabulário


In [9]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras

Algumas palavras do vocabulário:
['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo']


In [10]:
# Verificar o vetor de uma palavra específica
if 'filme' in model.wv:
    vetor_filme = model.wv['filme']
    print(f"\nVetor da palavra 'filme' (primeiras 10 dimensões):")
    print(vetor_filme[:10])
    print(f"Dimensionalidade do vetor: {len(vetor_filme)}")


Vetor da palavra 'filme' (primeiras 10 dimensões):
[-0.00053898  0.00023755  0.00510847  0.00900957 -0.00930602 -0.00711791
  0.00645901  0.00897323 -0.00500892 -0.00376375]
Dimensionalidade do vetor: 100


In [12]:
model.save("vasco.model")


In [15]:
vascao = Word2Vec.load("vasco.model")

In [16]:
print([nome for nome in list(api.info()['models'].keys())[:10]])

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']


In [17]:
modelo_pretreinado = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [18]:
# Carregar um modelo pré-treinado (word2vec-google-news-300)
# Nota: Isso pode demorar um pouco na primeira execução (download)
try:
    # Para português, você pode tentar 'word2vec-portuguese' se disponível
    # ou usar modelos do NILC: http://nilc.icmc.usp.br/embeddings
    modelo_pretreino = api.load("glove-wiki-gigaword-100")  # Mais rápido que word2vec-google-news-300
    print(f"Modelo pré-treinado carregado com {len(modelo_pretreino.key_to_index)} palavras")

    # Verificar palavras similares usando modelo pré-treinado
    if 'computer' in modelo_pretreino:
        similares = modelo_pretreino.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")
except Exception as e:
    print(f"Erro ao carregar modelo pré-treinado: {e}")
    print("Tente outro modelo ou prossiga sem esta parte")

Modelo pré-treinado carregado com 400000 palavras

Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [19]:
# Usando modelo pré-treinado para analogias (se disponível)
try:
    # Famosa analogia: rei - homem + mulher = rainha
    if all(word in modelo_pretreino for word in ['king', 'man', 'woman']):
        resultado = modelo_pretreino.most_similar(
            positive=['king', 'woman'],
            negative=['man'],
            topn=3
        )
        print("\nAnalogia: rei - homem + mulher =")
        for palavra, similaridade in resultado:
            print(f"{palavra}: {similaridade:.4f}")
except:
    print("Não foi possível realizar a operação de analogia com o modelo disponível")

# Podemos tentar outras analogias com nosso modelo treinado
# Por exemplo: bom - positivo + negativo = ruim
# (Dependendo do tamanho do corpus, pode não funcionar bem para modelos pequenos)



Analogia: rei - homem + mulher =
queen: 0.7699
monarch: 0.6843
throne: 0.6756


In [20]:
# Função para calcular similaridade entre pares de palavras
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados

# Pares de palavras para testar
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]

# Calcular similaridades
similaridades = calcular_similaridade(model, pares)

# Exibir resultados
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")



Similaridade entre pares de palavras:
filme - cinema: 0.1693534404039383
bom - ruim: Uma ou ambas as palavras não estão no vocabulário
ator - atuação: Uma ou ambas as palavras não estão no vocabulário
filme - protagonista: -0.11359719187021255


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))


In [21]:
import spacy

# Carregar modelo spaCy (pequeno)
try:
    nlp = spacy.load("pt_core_news_sm")  # Para português
    # Alternativa para inglês: nlp = spacy.load("en_core_web_sm")

    # Exemplo de texto
    texto = "O banco está cheio de dinheiro. Eu sentei no banco da praça."

    # Processar o texto
    doc = nlp(texto)

    # Examinar embeddings para cada ocorrência da palavra "banco"
    for token in doc:
        if token.text.lower() == "banco":
            contexto = doc[max(0, token.i-3):min(len(doc), token.i+4)]
            print(f"Contexto: {contexto}")
            print(f"Vetor (10 primeiras dimensões): {token.vector[:10]}")
            print(f"Dimensão do vetor: {len(token.vector)}")
            print("-" * 50)
except Exception as e:
    print(f"Erro ao carregar spaCy: {e}")
    print("Talvez seja necessário instalar os modelos com:")
    print("python -m spacy download pt_core_news_sm")
    print("ou")
    print("python -m spacy download en_core_web_sm")


Contexto: O banco está cheio de
Vetor (10 primeiras dimensões): [-2.761275    2.7341504   1.9384544   0.34338498  1.0899546  -4.300789
  1.5131965   3.7354984   1.1549678  -1.9115787 ]
Dimensão do vetor: 96
--------------------------------------------------
Contexto: Eu sentei no banco da praça.
Vetor (10 primeiras dimensões): [-2.8606548   3.2033174   4.2152634   2.3896182   4.4038973  -3.509808
  4.1706643   5.813535   -0.12308097 -0.5815916 ]
Dimensão do vetor: 96
--------------------------------------------------


In [22]:
from gensim.models import FastText

# Treinar modelo FastText
modelo_fasttext = FastText(
    sentences=textos_preprocessados,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# Testar com palavras que não estão no corpus
palavras_teste = ["filmagem", "cinematográfico", "atuações"]

print("\nEmbeddings para palavras fora do vocabulário (FastText):")
for palavra in palavras_teste:
    # Verificar se a palavra está no vocabulário original
    no_vocab = palavra in model.wv
    # Obter vetor do FastText (funciona mesmo para palavras fora do vocabulário)
    vetor = modelo_fasttext.wv[palavra]
    print(f"'{palavra}' (no vocabulário: {no_vocab})")
    print(f"Primeiras 5 dimensões do vetor: {vetor[:5]}")

    # Encontrar palavras similares
    similares = modelo_fasttext.wv.most_similar(palavra, topn=3)
    print(f"Palavras similares a '{palavra}':")
    for p, sim in similares:
        print(f"  {p}: {sim:.4f}")
    print()



Embeddings para palavras fora do vocabulário (FastText):
'filmagem' (no vocabulário: False)
Primeiras 5 dimensões do vetor: [-0.00071874  0.00085034  0.00017239 -0.0009094  -0.00061973]
Palavras similares a 'filmagem':
  gostei: 0.2725
  filme: 0.2633
  cenas: 0.2351

'cinematográfico' (no vocabulário: False)
Primeiras 5 dimensões do vetor: [ 0.00013222  0.00121597  0.00156423 -0.00238611  0.00105269]
Palavras similares a 'cinematográfico':
  cinematografia: 0.6115
  cinema: 0.4938
  trilha: 0.2005

'atuações' (no vocabulário: False)
Primeiras 5 dimensões do vetor: [ 0.00126086 -0.00013844  0.00066779  0.00134189 -0.00153145]
Palavras similares a 'atuações':
  atuação: 0.3585
  personagens: 0.2834
  roteiro: 0.1743

